In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class CurrencyDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Currency Detection")

        # Set the window dimensions to make it a bit bigger
        self.root.geometry("800x600")

        self.title_label = tk.Label(root, text="Currency Detection", font=("Helvetica", 16, "bold"))
        self.title_label.pack(pady=10)

        self.browse_button = tk.Button(root, text="Load Image", command=self.browse_image)
        self.browse_button.pack(padx=20, pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack()

        self.result_label = tk.Label(root, text="", font=("Helvetica", 14))
        self.result_label.pack()

        self.save_button = tk.Button(root, text="Save Image", command=self.save_image, state="disabled")
        self.save_button.pack(pady=10)

        self.reset_button = tk.Button(root, text="Reset", command=self.reset_interface)
        self.reset_button.pack()

        self.display_opencv_button = tk.Button(root, text="Display OpenCV Images", command=self.display_opencv_images)
        self.display_opencv_button.pack()

        self.currency_image = None
        self.result = ""
        self.opencv_images = []

    def browse_image(self):
        image_path = filedialog.askopenfilename()
        if image_path:
            self.detect_currency(image_path)

    def detect_currency(self, image_path):
        self.currency_image = cv2.imread(image_path)

        # Preprocessing
        gray_image = cv2.cvtColor(self.currency_image, cv2.COLOR_BGR2GRAY)
        blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

        # Simple Edge Detection
        edges = cv2.Canny(blurred_image, threshold1=30, threshold2=100)

        # Feature Extraction (Contour Detection)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        largest_contour = max(contours, key=cv2.contourArea)
        contour_area = cv2.contourArea(largest_contour)

        # Decision Making
        threshold_area = 5000  # Adjust this threshold as needed
        if contour_area > threshold_area:
            self.result = "Genuine Currency"
        else:
            self.result = "Fake Currency"

        # Display the result
        self.result_label.config(text="Result: " + self.result)

        # Display the original image with results
        self.display_image(self.currency_image, largest_contour)
        self.save_button.config(state="normal")

        # Generate and store OpenCV images
        self.generate_opencv_images()

    def display_image(self, image, contour):
        result_image = image.copy()
        cv2.drawContours(result_image, [contour], -1, (0, 255, 0), 2)

        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        result_image = Image.fromarray(result_image)
        result_image.thumbnail((600, 400))
        img = ImageTk.PhotoImage(image=result_image)
        self.image_label.config(image=img)
        self.image_label.image = img

    def save_image(self):
        if self.currency_image is not None:
            save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg")])
            if save_path:
                cv2.imwrite(save_path, self.currency_image)

    def reset_interface(self):
        self.currency_image = None
        self.result = ""
        self.result_label.config(text="")
        self.image_label.config(image="")
        self.save_button.config(state="disabled")

    def generate_opencv_images(self):
        # Generate OpenCV images
        img_median = cv2.medianBlur(self.currency_image, 3)
        gray = cv2.cvtColor(img_median, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 60, 180)
        th500 = cv2.adaptiveThreshold(edges, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 10)
        self.opencv_images = [self.currency_image, img_median, gray, edges, th500]

    def display_opencv_images(self):
        # Display OpenCV images using cv2.imshow
        for i, opencv_image in enumerate(self.opencv_images):
            cv2.imshow(f'OpenCV Image {i}', opencv_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == "__main__":
    root = tk.Tk()
    app = CurrencyDetectionApp(root)
    root.mainloop()
